# Resposta às perguntas do arquivo "perguntas_sql.md"

Este *notebook* tem como objetivo responder às questões 1 a 10 utilizando Python, com ênfase no uso da biblioteca `pandas`. Considerando que a base principal contém mais de 10 milhões de registros, algumas consultas serão realizadas com o método `read_sql()` do pacote `basedosdados`, a fim de obter dados minimamente tratados diretamente do BigQuery.

Embora o método `read_sql()` permita responder diretamente às questões utilizando comandos SQL, o foco deste teste técnico é demonstrar proficiência com o `pandas`. Portanto, as consultas SQL foram propositalmente simplificadas para que o tratamento dos dados relevantes às perguntas seja realizado majoritariamente em Python.

O notebook está dividido em duas seções:
- **Respostas às perguntas 1 a 5**
- **Respostas às perguntas 6 a 10**

No início de cada seção, há *chunks* de código que devem ser executados previamente. Esses blocos geralmente incluem consultas que retornam bases de dados "brutas" necessárias para as questões da seção ou procedimentos que evitam redundâncias — como tratamentos recorrentes utilizados por mais de uma pergunta.

**Portanto, para rodar qualquer uma das perguntas de uma seção, é obrigatório executar:**
1. O primeiro bloco geral do notebook (logo abaixo deste texto), que carrega bibliotecas e funções auxiliares.
2. Os blocos de código logo após o título da seção correspondente (iniciados com `##`), que trazem as bases ou preparações comuns às perguntas daquela seção.

Cada subseção de pergunta (marcadas com `###`) contém *chunks* independentes entre si. Ou seja, **você não precisa rodar o código da pergunta 6 para executar a pergunta 7** — desde que os blocos gerais e os da seção já tenham sido executados previamente.

In [ ]:
# Libs utilizadas
import pandas as pd
import basedosdados as bd

# billing id para o basedosdados
bill_id = "teste-tecnico-pcrj"

# Funções que serão utilizadas ao longo do notebook

def operacao_agrupada_chamados(df: pd.DataFrame,
                               colunas: list,
                               operacao: str = 'sum'):
    """
    Função para somar chamados agrupando por colunas especificadas. Utilizada no conjunto de respostas 1-5.
    
    Args:
        df (pd.DataFrame): DataFrame com os dados.
        colunas (list): Lista com as colunas que serão utilizadas para agrupar os dados.
    
    Returns:
        pd.DataFrame: DataFrame com a soma dos chamados pelos grupos. A última coluna sempre será o total de chamados.
    """
    df_agrupado = (
        df.groupby(colunas)
        .agg({'total_chamados': f'{operacao}'})
        .reset_index()
        .sort_values('total_chamados', ascending=False))
    
    return df_agrupado

## Respostas às perguntas 1-5

In [103]:
# preparando objetos para as questões 1-5

sql_1 = """
SELECT 
  COUNT(id_chamado) AS total_chamados,
  DATE(data_inicio) AS data_abertura,
  tipo,
  subtipo,
  id_bairro
FROM `datario.adm_central_atendimento_1746.chamado` 
WHERE DATE(data_inicio)= '2023-04-01'
GROUP BY DATE(data_inicio), tipo, subtipo, id_bairro
"""

sql_2 = """
SELECT 
  id_bairro,
  nome AS nome_bairro,
  subprefeitura
FROM `datario.dados_mestres.bairro`
"""

df_base_1 = bd.read_sql(sql_1, billing_project_id=bill_id)
df_bairros = bd.read_sql(sql_2, billing_project_id=bill_id)

Downloading: 100%|██████████|
Downloading: 100%|██████████|


### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [104]:
total_chamados_soma = df_base_1['total_chamados'].sum()

print(f'R: O total de chamados abertos no dia 01/04/2023 é de {total_chamados_soma} chamados.')

R: O total de chamados abertos no dia 01/04/2023 é de 1903 chamados.


### 2. Qual é o tipo de chamado que teve mais chamados abertos no dia 01/04/2023?

In [105]:
chamados_por_dia = operacao_agrupada_chamados(df_base_1, ['tipo'])

print(f'R: O tipo de chamado com maior quantidade de chamados abertos no dia 01/04/2023 é {chamados_por_dia.iloc[0,0]} com {chamados_por_dia.iloc[0,1]} chamados.')

R: O tipo de chamado com maior quantidade de chamados abertos no dia 01/04/2023 é Estacionamento irregular com 373 chamados.


### 3. Quais os nomes dos bairros que mais tiveram chamados abertos nesse dia?

In [106]:
df_base_bairros = df_base_1.merge(df_bairros, on='id_bairro', how='left')

chamados_por_bairros = operacao_agrupada_chamados(df_base_bairros, ['nome_bairro'])

top_n = 3

print(f'O top {top_n} bairros com maior quantidade de chamados abertos no dia 01/04/2023 foram:')
for i in range(top_n):
    print(f'{i+1} - {chamados_por_bairros.iloc[i,0]} com {chamados_por_bairros.iloc[i,1]} chamados.')

O top 3 bairros com maior quantidade de chamados abertos no dia 01/04/2023 foram:
1 - Campo Grande com 124 chamados.
2 - Tijuca com 96 chamados.
3 - Barra da Tijuca com 60 chamados.


### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [107]:
df_base_bairros = df_base_1.merge(df_bairros, on='id_bairro', how='left')

chamados_por_subprefeitura = operacao_agrupada_chamados(df_base_bairros, ['subprefeitura'])

print(f'A subprefeitura com maior quantidade de chamados abertos no dia 01/04/2023 foi a {chamados_por_subprefeitura.iloc[0,0]} com {chamados_por_subprefeitura.iloc[0,1]} chamados.')

A subprefeitura com maior quantidade de chamados abertos no dia 01/04/2023 foi a Zona Norte com 534 chamados.


### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece? 

In [108]:
df_bairro_nulo = df_base_1.loc[df_base_1['id_bairro'].isnull()]
df_bairro_nulo_por_tipo = operacao_agrupada_chamados(df_bairro_nulo,
                                                 ['tipo'])

df_bairro_nulo_por_tipo.head(10)

,tipo,total_chamados
33,Ônibus,50
1,Atendimento ao cidadão,18
4,Clínicas da Família (CF),13
30,Sistema Nacional de Regulação (SISREG),6
21,Ouvidoria SMTR,5
14,Iluminação Pública,3
13,Hospitais (HM),3
8,Diversos - Comlurb,3
22,Postos de Saúde (PS),2
6,Conservação de vias,2


R: Sim, existiram chamados que não foram associados à nenhum id_bairro e, portanto, à nenhuma subprefeitura. Avaliando os principais tipos de chamados que possuem esta característica, é possível verificar que essas questões não dependem do espaço geográfico do cidadão que as origina. Por exemplo, chamados em relação aos ônibus não necessariamente serão questões de um só bairro, visto que existem ônibus (a grande maioria) cujo trajeto perpassa múltiplos bairros. Atendimento ao cidadão é outro caso, visto que dúvidas, sugestões, aviso de erros ou problemas com serviços digitais da prefeitura podem não ser ligados ao local em que o cidadão está fazendo o chamado. Ou seja, a maior parte dessas questões não são dependentes diretamente da localização.

## Respostas às perguntas 6-10

In [109]:
# Preparando os data frames básicos para as questões 6-10

sql_22_23 ="""
SELECT COUNT(id_chamado) AS total_chamados,
    DATE(data_inicio) AS data_abertura,
    subtipo
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
GROUP BY data_abertura, subtipo
"""

sql_eventos = """
SELECT *
FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
"""

df_22_23 = bd.read_sql(sql_22_23, billing_project_id=bill_id)
df_eventos = bd.read_sql(sql_eventos, billing_project_id=bill_id)

Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [110]:
# Precisamos limpar a base de dados de eventos para as próximas questões
df_eventos_clean = (
    # Filtrando apenas eventos com data_inicial e data_final válidas e selecionando as colunas de interesse
    df_eventos.loc[
        df_eventos['data_inicial'].notnull() & df_eventos['data_final'].notnull(),
        ['evento', 'data_inicial', 'data_final']
    ]
    .assign(
        # Convertendo as datas ao formato datetime
        data_inicial =  lambda x: pd.to_datetime(x['data_inicial']),
        data_final   =    lambda x: pd.to_datetime(x['data_final']),
        # Expandindo as datas dos eventos
        data_evento  =   lambda x: x.apply(
            lambda y: pd.date_range(y['data_inicial'], y['data_final']), axis=1
            )
        )
    .explode('data_evento')
    .reset_index(drop=True)
)
df_eventos_clean

,evento,data_inicial,data_final,data_evento
0,Carnaval,2024-02-10,2024-02-14,2024-02-10
1,Carnaval,2024-02-10,2024-02-14,2024-02-11
2,Carnaval,2024-02-10,2024-02-14,2024-02-12
3,Carnaval,2024-02-10,2024-02-14,2024-02-13
4,Carnaval,2024-02-10,2024-02-14,2024-02-14
5,Carnaval,2023-02-18,2023-02-21,2023-02-18
6,Carnaval,2023-02-18,2023-02-21,2023-02-19
7,Carnaval,2023-02-18,2023-02-21,2023-02-20
8,Carnaval,2023-02-18,2023-02-21,2023-02-21
9,Rock in Rio,2022-09-02,2022-09-04,2022-09-02


In [111]:
# Precisamos, agora, usar o read_sql do bd para pegar os chamados abertos nos dias de evento
lista_datas = df_eventos_clean['data_evento'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d')).to_list()

# String com o conjunto de datas do df_eventos_clean
str_datas = '(' + ', '.join(f"'{data}'" for data in lista_datas) + ')'

query_evento_chamados = f"""
SELECT COUNT(id_chamado) AS total_chamados,
    DATE(data_inicio) AS data_abertura,
    subtipo
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) IN {str_datas}
GROUP BY data_abertura, subtipo 
"""

# novo df com os chamados dos eventos
df_eventos_chamados = bd.read_sql(query_evento_chamados, billing_project_id=bill_id)
df_eventos_chamados = (
    df_eventos_chamados.assign(
        data_abertura = lambda x: pd.to_datetime(x['data_abertura'])
    ).merge(
        df_eventos_clean,
        left_on='data_abertura',
        right_on='data_evento',
        how='left'
    )
)

Downloading: 100%|██████████|


### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [112]:
# filtrando os subtipos que contém 'sossego'
df_22_23_sossego = df_22_23.loc[df_22_23['subtipo'].str.contains('sossego', case=False, na=False)]

subtipo_perturbacao_soma = operacao_agrupada_chamados(df_22_23_sossego, ['subtipo'])

print('Entre o período solicitado, não há um subtipo exato de "Perturbação do sossego". Assim, escolhi os subtipos com nomes próximos, a saber:')
for i in range(subtipo_perturbacao_soma.shape[0]):
    print(f'{subtipo_perturbacao_soma.iloc[i,0]} com {subtipo_perturbacao_soma.iloc[i,1]} chamados.')
print(f'\nA soma dos chamados com subtipos próximos de "Perturbação do sossego" é de {subtipo_perturbacao_soma["total_chamados"].sum()} chamados.')

Entre o período solicitado, não há um subtipo exato de "Perturbação do sossego". Assim, escolhi os subtipos com nomes próximos, a saber:
Fiscalização de perturbação do sossego com 50368 chamados.
Informações sobre Perturbação do Sossego com 11590 chamados.

A soma dos chamados com subtipos próximos de "Perturbação do sossego" é de 61958 chamados.


**OBSERVAÇÃO:** Considerando que este subtipo será utilizado nas próximas questões, é importante ressaltar que a nomenclatura exata pode variar ao longo do tempo. Assim, utilizarei ambas as classificações apresentadas acima e quaisquer outras que pareçam coerentes com o subtipo.

### 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [113]:
df_eventos_chamados = df_eventos_chamados.loc[df_eventos_chamados['subtipo'].str.contains('sossego', case=False, na=False)]

print(df_eventos_chamados)

      total_chamados data_abertura                                   subtipo  \
18                51    2022-12-30    Fiscalização de perturbação do sossego   
89                77    2023-02-18    Fiscalização de perturbação do sossego   
90                91    2023-02-19    Fiscalização de perturbação do sossego   
92                46    2023-02-21    Fiscalização de perturbação do sossego   
123              106    2022-09-02    Fiscalização de perturbação do sossego   
124               91    2022-09-08    Fiscalização de perturbação do sossego   
125              126    2022-09-03    Fiscalização de perturbação do sossego   
227               41    2024-02-10  Informações sobre Perturbação do Sossego   
379               37    2023-12-30  Informações sobre Perturbação do Sossego   
419               22    2024-02-14  Informações sobre Perturbação do Sossego   
697              180    2022-09-11    Fiscalização de perturbação do sossego   
820               26    2024-02-12  Info

### 8. Quantos chamados desse subtipo foram abertos em cada evento ?

In [114]:
total_chamados_subtipo = operacao_agrupada_chamados(df_eventos_chamados, ['evento'])

print(f'Número de chamados abertos com subtipos próximos de "Perturbação do sossego": ')
for i in range(total_chamados_subtipo.shape[0]):
    print(f'{total_chamados_subtipo.iloc[i,0]} com {total_chamados_subtipo.iloc[i,1]} chamados.')

Número de chamados abertos com subtipos próximos de "Perturbação do sossego": 
Rock in Rio com 946 chamados.
Carnaval com 391 chamados.
Réveillon com 239 chamados.


### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo ?

In [115]:
df_eventos_media = operacao_agrupada_chamados(df_eventos_chamados, ['evento'], 'mean')

print(f'O evento com maior média de chamados abertos foi {df_eventos_media.iloc[0,0]} com cerca de {df_eventos_media.iloc[0,1]:.0f} chamados.')

O evento com maior média de chamados abertos foi Rock in Rio com cerca de 135 chamados.


### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo consideranto todo o período de 01/01/2022 até 31/12/2023.

In [116]:
df_eventos_media = operacao_agrupada_chamados(df_eventos_chamados, ['evento'], 'mean')

df_22_23_sossego = df_22_23.loc[df_22_23['subtipo'].str.contains('sossego', case=False, na=False)]
df_22_23_sossego = operacao_agrupada_chamados(df_22_23_sossego, ['data_abertura'])

media_22_23 = df_22_23_sossego['total_chamados'].mean()

print(f'A média de chamados abertos com subtipos próximos de "Perturbação do sossego" entre 2022 e 2023 foi de {media_22_23:.0f} chamados.')
print(f'A média de chamados abertos com subtipos próximos de "Perturbação do sossego" durante os eventos foram:')
for i in range(df_eventos_media.shape[0]):
    print(f'{df_eventos_media.iloc[i,0]} com {df_eventos_media.iloc[i,1]:.0f} chamados.')

A média de chamados abertos com subtipos próximos de "Perturbação do sossego" entre 2022 e 2023 foi de 85 chamados.
A média de chamados abertos com subtipos próximos de "Perturbação do sossego" durante os eventos foram:
Rock in Rio com 135 chamados.
Carnaval com 43 chamados.
Réveillon com 40 chamados.


R: Entre 2022 e 2023, os cariocas abriram cerca de 85 chamadas por dia em média do tipo "Perturbação do sossego", sendo maior que o número de chamadas médias do carnaval e do Réveillon. Por outro lado, o Rock in Rio superou o número de reclamações média diárias, sendo quase 60% maior que a média total do período de 2022-2023.